<a href="https://colab.research.google.com/github/DANU011/PyDataAnalytics/blob/main/%EC%98%81%ED%99%94_%EA%B4%80%EA%B0%9D%EC%88%98_%EC%98%88%EC%B8%A1/notebook/%EC%98%81%ED%99%94_%EA%B4%80%EA%B0%9D%EC%88%98_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. pandas를 이용해 csv 파일 읽어오기

In [2]:
import pandas as pd

train = pd.read_csv('movies_train.csv')
test = pd.read_csv('movies_test.csv')
submission = pd.read_csv('submission.csv')

## 2. 결측치를 평균값으로 채운 train_fill 데이터셋 생성하기

In [3]:
train_mean = train['dir_prev_bfnum'].mean()

train_fill = train.fillna(train_mean)
test_fill = test.fillna(train_mean)
train_fill

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,1.050443e+06,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1.161602e+06,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,2.207752e+05,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,2.389400e+04,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.000000e+00,1,251,2,483387
...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,(주)NEW,드라마,2014-08-13,111,청소년 관람불가,심성보,3.833000e+03,1,510,7,1475091
596,파파로티,(주)쇼박스,드라마,2013-03-14,127,15세 관람가,윤종찬,4.960610e+05,1,286,6,1716438
597,살인의 강,(주)마운틴픽쳐스,공포,2010-09-30,99,청소년 관람불가,김대현,1.050443e+06,0,123,4,2475
598,악의 연대기,CJ 엔터테인먼트,느와르,2015-05-14,102,15세 관람가,백운학,1.050443e+06,0,431,4,2192525


## 3. 영화 관객 수 모델 학습

In [4]:
formula = '''
box_off_num ~ (time) + (dir_prev_bfnum) + (dir_prev_num) + (num_staff) + (num_actor)
'''
'''
Python의 통계 분석 라이브러리 중 하나인 statsmodels에서 사용될 수 있는 모델을 정의하는데 사용된다.
box_off_num: 종속 변수, 예측하려는 대상.
time, dir_prev_num, num_staff, num_actor: 독립 변수, 종속 변수를 예측하는 데 사용되는 설명 변수.
따라서 이 모델은 time, dir_prev_num, num_staff, num_actor라는 변수들을 사용하여 box_off_num이라는 종속 변수를 예측하는 회귀 모델.
''';

import statsmodels.api as sm

model = sm.OLS.from_formula(formula, data = train_fill) # statsmodels의 OLS 클래스를 사용하여 회귀 모델을 정의
model = model.fit() # 모델 학습
predict = model.predict(test_fill) # 모델 예측

## 4. 예측된 영화 관객 수를 submission 파일에 추가

In [5]:
submission['box_off_num'] = predict

submission.head()

,title,box_off_num
0,용서는 없다,1.670730e+06
1,아빠가 여자를 좋아해,1.412505e+06
2,하모니,2.821661e+06
3,의형제,2.019305e+06
4,평행 이론,1.628432e+06


## 5. 데이터프레임(DataFrame)을 CSV 파일로 저장

In [9]:
submission.to_csv('submission.csv', index=False)